In [2]:
import xpress as xp
import networkx as nx


# TODO:
# Incorporate all_cycles from Julie's code
# Figure out why the call back thing does not work 
# Test on a small dataset

In [48]:
# The following cell defines the separation algorithm and the call back function to be used in Xpress. 

def separation_algorithm(model, y, f_i, nodes, altruistic_donors):
    """
    Implements the separation algorithm for cut-set constraints based on the supplementary information.
    
    >>> Inputs:
    model:                XPRESS optimization model
    y:                    Dictionary where keys are edge tuples (u, v) and values are their solution values i.e., y[e] gotten via current_sol[0] from above
    f_i:                  Dictionary where keys are nodes and values are the solver solution for f_i[v] gotten via current_sol[2] from above
    nodes:                List of all nodes (pre-defined)
    altruistic_donors:    List of all altruistic donor nodes (pre-defined)

    >>> Outputs:
    N/A, this function just finds violated constraints and adds them to the model.
    """
    
    # Construct the directed graph G = (V, E, w) based on supplementary information definition 
    G = nx.DiGraph()
    for node in nodes:
        G.add_node(node)

    # Add edges with weights from y (solution values)
    for (u, v), sol_value in y.items():
        G.add_edge(u, v, weight=sol_value)

    # Add super source 'SUPER' and connect to all NDDs
    super_source = "SUPER"
    G.add_node(super_source)
    for u in altruistic_donors:
        G.add_edge(super_source, u, weight=1)

    # Initialize output of violated constraints 
    violated_constraints = []
    
    # Solve the max-flow min-cut problem for each node with f_i[v] > 0
    for v, flow_in in f_i.items():
        if flow_in > 0:
            
            # Compute min-cut using the function from networkx
            cut_value, (S, Not_S) = nx.minimum_cut(G, super_source, v, capacity="weight")

            # If the cut weight is less than flow_in, add a violated constraint
            if cut_value < flow_in:

                # Define delta_minus_S in this case
                delta_minus_S = [(u, v) for u in S for v in Not_S if G.has_edge(u, v)]

                # Create the constraint and add it to the list of constraints
                constraint = xp.Sum(y[e] for e in delta_minus_S) >= flow_in
                violated_constraints.append(constraint)
                print(f"Added violated constraint for node {v}: Cut separating {S} and {Not_S}")

    # Note: This version returns violated constraints after doing all the checking; A possible 
    #       extension is to try if its faster to return once we found a single violated constraint. 
    return violated_constraints




def separation_callback(model):
    """
    Callback function for XPRESS to dynamically add cut constraints.
    
    >>> Input:
    model: XPRESS optimization model.

    >>> Output:
    N/A, this function just incorporates the separation into the solver
    """
    # Retrieve current relaxed values for y and f
    
    print("I'm trying!")
    current_sol_y = {e: model.getSolution(y[e]) for e in edges}
    current_sol_f_i = {v: model.getSolution(f_i[v]) for v in nodes}

    
    
    # Run the separation algorithm to find violated constraints
    violated_constraints = separation_algorithm(model, current_sol_y, current_sol_f_i, nodes, altruistic_donors)
    
    # # If there are violated constraints, add them dynamically
    # for constraint in violated_constraints:
    #     # model.addConstraint(constraint)
    #     return []
    return model.getSolution()

In [49]:
# What we need to feed this model is the following:
# 1- List of donor-receipient pairs
# 2- List of altruisitic donors
# 3- List of nodes = donor-receipient pairs + altruisitic donors
# 4- Dictionary of edges with key ("node1","node2") and value is the weight of the edge
# 5- Dictionary of cycles with key (edge1,edge2,edge3,...) and value is the weight of the cycle i.e., sum of edges in the cycle up to and including length k
pairs = ["P1", "P2", "P3"] 
altruistic_donors = ["NDD1"]
nodes = pairs + altruistic_donors
edges = {("NDD1", "P1"): 10, 
         ("NDD1", "P2"): 8,
         ("P1", "P3"): 8, 
         ("P2", "P3"): 9,
         ("P3", "P1"): 6,
         ("P1", "P2"): 6}
all_cycles = {}






# Define the optimization model
model = xp.problem()



# Decision variables
y = {e: xp.var(vartype=xp.binary, name=f"y_{e}") for e in edges}  # Edge selection
z = {c: xp.var(vartype=xp.binary, name=f"z_{c}") for c in all_cycles}  # Cycle selection
f_i = {v: xp.var(vartype=xp.binary) for v in nodes}  # Flow in decision variable
f_o = {v: xp.var(vartype=xp.binary) for v in nodes}  # Flow out decision variable

model.addVariable(list(y.values()) + list(z.values())+ list(f_i.values()) + list(f_o.values()))



# Define the objective function which is to maximize sum of weights of selected edges and cycles
model.setObjective(xp.Sum(edges[e] * y[e] for e in edges) + xp.Sum(all_cycles[c] * z[c] for c in all_cycles), sense=xp.maximize)




### Constraints:

# 1. Defining f_i and f_o i.e., the incoming and outgoing kidneys to a node v
for v in nodes:
    model.addConstraint(xp.Sum([y[e] for e in edges if e[1] == v]) == f_i[v])
    model.addConstraint(xp.Sum([y[e] for e in edges if e[0] == v]) == f_o[v])

# 2. Ensure that if a node is in an actived cycle, the associated edges that involve it are turned off
for v in pairs:
    model.addConstraint(f_o[v] + xp.Sum([z[c] for c in all_cycles if v in c]) <= f_i[v] + xp.Sum([z[c] for c in all_cycles if v in c]))
    model.addConstraint(f_i[v] + xp.Sum([z[c] for c in all_cycles if v in c]) <= 1)

# 3. Ensure that altruistic donors donate at most one kidney  
for v in altruistic_donors:
    model.addConstraint(f_o[v] <= 1)

# 4. The hard and confusing constraint... This version enumerates all possible edges. I just added for completeness
# i, k = 2, 3
# subsets = [subset for j in range(i, k+1) for subset in itertools.combinations(pairs, j)]
# for S in subsets:
#     for v in S:
#          model.addConstraint(xp.Sum([y[e] for e in edges if (e[1] in S and e[0] not in S)]) >= f_i[v])



# Add the call back function to detect and add violated constraints during LP relaxation
model.addcbpreintsol(separation_callback, separation_algorithm, 3)

# Solve the model
model.solve()

#
print(model.getProbStatusString())

# Get the current solution
current_sol = ({e: model.getSolution(y[e]) for e in edges}, {c: model.getSolution(z[c]) for c in all_cycles}, {v: model.getSolution(f_i[v]) for v in nodes})

# Extract and print solution
print("Objective Value:", model.getObjVal())
selected_edges = [e for e in edges if model.getSolution(y[e]) > 0.5]
selected_cycles = [c for c in all_cycles if model.getSolution(z[c]) > 0.5]
print("Selected Edges:", selected_edges)
print("Selected Cycles:", selected_cycles)

FICO Xpress v9.4.2, Hyper, solve started 15:37:55, Feb 12, 2025
Heap usage: 394KB (peak 394KB, 135KB system)
Maximizing MILP noname using up to 14 threads and up to 15GB memory, with these control settings:
OUTPUTLOG = 1
NLPPOSTSOLVE = 1
XSLP_DELETIONCONTROL = 0
XSLP_OBJSENSE = -1
Original problem has:
        15 rows           14 cols           30 elements        14 entities
Presolved problem has:
         7 rows            9 cols           18 elements         9 entities
Presolve finished in 0 seconds
Heap usage: 425KB (peak 447KB, 135KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  Objective      [min,max] : [ 6.00e+00,  1.00e+01] / [ 6.00e+00,  1.00e+01]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 8.7GB
User interrupt (7) triggered.
 *** S

SolverError: ?557 Error: Integer solution is not available

In [17]:
################### Debugging separation algorithm implementation 



# ### Example 1: 

# # Example solution for y (edges)
# y = {("NDD1", "P1"): 0.8, ("P1", "P2"): 0.6, ("P2", "P3"): 0.4, ("P3", "P4"): 0.3, ("NDD2", "P5"): 0.7}

# # Example solution for f (node flow)
# f_i = {"P1": 0.8, "P2": 0.6, "P3": 0.4, "P4": 0.3, "P5": 0.7}

# # Nodes in the graph
# nodes = ["NDD1", "P1", "P2", "P3", "P4", "P5", "NDD2"]

# # Altruistic donors
# altruistic_donors = ["NDD1","NDD2"]




# ### Example 2: 

# # Example LP solution for y (edges forming a cycle)
# y = {("P1", "P2"): 0.9, ("P2", "P3"): 0.8, ("P3", "P4"): 0.7, ("P4", "P1"): 0.6}  

# # Example LP solution for f (node flow)
# f_i = {"P1": 0.6, "P2": 0.9, "P3": 0.8, "P4": 0.7}  

# # Nodes in the graph
# nodes = ["P1", "P2", "P3", "P4"]

# # Altruistic donors
# altruistic_donors = []




# ### Example 3: 

# # Example LP solution for y (edges forming a cycle)
# y = {("P1", "P2"): 1, ("P2", "P3"): 1, ("P3", "P4"): 1, ("P4", "P1"): 1}  

# # Example LP solution for f (node flow)
# f_i = {"P1": 1, "P2": 1, "P3": 1, "P4": 1}  

# # Nodes in the graph
# nodes = ["P1", "P2", "P3", "P4"]

# # Altruistic donors
# altruistic_donors = []


### Example 4: 

# Example LP solution for y (edges forming a cycle)
y = current_sol[0]  

# Example LP solution for f (node flow)
f_i = current_sol[2] 

pairs = ["P1", "P2", "P3"] 
altruistic_donors = ["NDD1"]
nodes = pairs + altruistic_donors


def separation_algorithm(model, y, f_i, nodes, altruistic_donors):
    """
    Implements the separation algorithm for cut-set constraints based on the supplementary information.
    
    >>> Inputs:
    model:                XPRESS optimization model
    y:                    Dictionary where keys are edge tuples (u, v) and values are their solution values i.e., y[e] gotten via current_sol[0] from above
    f_i:                  Dictionary where keys are nodes and values are the solver solution for f_i[v] gotten via current_sol[2] from above
    nodes:                List of all nodes (pre-defined)
    altruistic_donors:    List of all altruistic donor nodes (pre-defined)

    >>> Outputs:
    N/A, this function just finds violated constraints and adds them to the model.
    """
    # Construct the directed graph G = (V, E, w) based on supplementary information definition 
    G = nx.DiGraph()
    for node in nodes:
        G.add_node(node)

    # Add edges with weights from y (solution values)
    for (u, v), sol_value in y.items():
        G.add_edge(u, v, weight=sol_value)

    # Add super source 'SUPER' and connect to all NDDs
    super_source = "SUPER"
    G.add_node(super_source)
    for u in altruistic_donors:
        G.add_edge(super_source, u, weight=1)

    # Solve the max-flow min-cut problem for each node with f_i[v] > 0
    for v, flow_in in f_i.items():
        if flow_in > 0.001:
            
            # Compute min-cut using the function from networkx
            cut_value, (S, Not_S) = nx.minimum_cut(G, super_source, v, capacity="weight")

            # If the cut weight is less than flow_in, add a violated constraint
            if cut_value < flow_in:
                
                print(v, cut_value, flow_in)
                delta_minus_S = [(u, v) for u in S for v in Not_S if G.has_edge(u, v)]

                # model.addConstraint(xp.Sum(y[e] for e in delta_minus_S) >= flow_in)
                print(f"Added violated constraint for node {v}: Cut separating {S} and {Not_S}")

                # Leave the function to just add this one constraint and re-solve
                return True

    # If we reach here it means there are no more cuts so return False
    return False





print("Algorithm executing...")
separation_algorithm("model_placeholder", y, f_i, nodes, altruistic_donors)
print("Algorithm executed!")



Algorithm executing...
Algorithm executed!


In [ ]:
# Note: This version is old but it could be useful for the recursive algorithm

### High-level structure to add violated constraints iteratively

while True:
    # Find a violated constraint - not sure how to do that?
    violated_constraint_output = find_violated_constraint(current_sol)
    
    # Check if no more violated constraints then stop
    if violated_constraint is "No more violated constraints!":
        break  

    # The output should be the set S and the list of edges in delta_minus_S
    S, delta_minus_S = violated_constraint_output
    
    # Add the constraint
    for v in S:
        model.addConstraint(xp.Sum(y[e] for e in delta_minus_S) >= f_i[v])
    
    # Re-optimize with new constraint
    model.solve()
    
    # Update solution with new values
    current_sol = ({e: model.getSolution(y[e]) for e in edges}, {c: model.getSolution(z[c]) for c in all_cycles})




### Print Final Solution

final_solution = ({e: model.getSolution(y[e]) for e in edges}, {c: model.getSolution(z[c]) for c in all_cycles})
print("Final Edge Selections:", final_solution)